In [2]:
import re
import pandas as pd
from string import ascii_letters as eng_letters
from string import digits
remove_digits = str.maketrans('', '', digits)
from farasa.stemmer import FarasaStemmer
from farasa.segmenter import FarasaSegmenter
import pprint
import time
stemmer = FarasaStemmer(interactive = True)

[2020-11-15 08:17:16,362 - DEBUG]: perform system check...
[2020-11-15 08:17:16,363 - DEBUG]: check java version...
[2020-11-15 08:17:16,599 - DEBUG]: Your java version is 1.8 which is compatiple with Farasa 
[2020-11-15 08:17:16,600 - DEBUG]: check toolkit binaries...
[2020-11-15 08:17:16,602 - INFO]: Dependencies seem to be satisfied..
[2020-11-15 08:17:16,602 - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2020-11-15 08:17:16,603 - INFO]: initializing [STEM] task in INTERACTIVE mode...
[2020-11-15 08:17:18,897 - INFO]: task [STEM] is initialized interactively.


In [3]:
import sys 
import stop_words_handler
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
def clean_text(sentence, hard=False,stem=True):
    words = []

    if not hard:
        punc = '''</>~%#`÷×؛_()*&^%][ـ/"؟'{}~¦+|!”…“–ـ'''
        punc = punc + eng_letters
    else:
        english_punctuations = string.punctuation
        arabic_punctuations = '''`÷×؛_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
        punc = english_punctuations + arabic_punctuations

    for w in sentence.split():

        # Cleaning Numbers
        w = w.translate(remove_digits)

        # Removing the punctuations
        for ch in punc:
            w = w.replace(ch, '')

        if any(sep in w for sep in ['.', '،', ':']):
            w = w[0:-1] + ' ' + w[-1]

        if not re.search(r'[a-zA-Z]', w):
            ## here we should handle ب and بال 
            if stem:
                words.append(stemmer.stem(w))
            else:
                words.append(w)

    return ' '.join(words)

In [5]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [6]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [7]:
yaqutz_pd = pd.read_excel('data/structured/FullYaqut.xlsx', sheet_name='ALL')



In [8]:

stop_words = stop_words_handler.Stopwords_Handler()
print(stop_words.unvoweled_df.columns)
stop_words = stop_words.unvoweled_df['@unvoweledform'].to_list()

Index(['@unvoweledform', '@ids'], dtype='object')


In [62]:
len(stop_words)

325

In [10]:
yaqutz_pd.columns

Index(['#', 'PlaceName', 'Description', 'Latitude', 'Longitude', 'Type1',
       'Type2', 'TypeFull', 'PlaceType', 'TypeEng', 'TrHebrew', 'TrEnglish',
       'wikidataID', 'WikiNameEng', 'WikiNamesAlt', 'ModernCountry',
       'ModernClass', 'elevation', 'geonames_id', 'wikilat', 'wikilong',
       'TurLat', 'TurLong', 'TurNameTr', 'TurNameEng', 'TurNameAr', 'TurType',
       'TurRegionEng', 'TurRegionTr'],
      dtype='object')

In [11]:
yaqutz_pd=yaqutz_pd[['PlaceName','Description']]
yaqutz_pd['Clean_desc'] = yaqutz_pd['Description'].apply(lambda x : clean_text(x,stem=False))

In [12]:
yaqutz_pd['Stem_desc'] = yaqutz_pd['Description'].apply(lambda x : clean_text(x))

In [13]:
docs = yaqutz_pd['Clean_desc'].tolist()

In [14]:
stem_docs = yaqutz_pd['Stem_desc'].tolist()
stem_docs

[' جمع بئر . قال في جمع بئر بئار أبار : موضع بين أجفر يد ، على  خمس ميل من أجفر . بئر أيض غير مضاف : كورة من كور واسط .',
 ' فتح همزة بعد ألف باء موحد مفتوح جيم : موضع في بلد معجم نسب  إلى أبو عبد الله محمد ابن محموية ابن مسلم آبجي ، روى عن أب غير ،  أخرج حاكم حديث ، لا درى هوى نسبة إلى آبه زيد جيم نسب ، كما  قال في نسبة إلى رمى أرمجي إلى خونى خونجي ، أم لا ، الله أعلم .',
 ' فتح همزة سكون ألف ضم باء موحد وراء : قرية من قرية سجستان ،  نسب إلى أبو حسن محمد ابن حسين ابن إبراهيم ابن عاصم آبري ، شيخ من  إمام حديث ، ل كتاب نفيس كبير في خبر إمام أب عبد الله محمد ابن  إدريس شافعي ، رضي الله عن ، أجاد في كل إجادة ، كان رحل إلى مصر  شام حجاز عراق خراسان ، روى عن أب بكر ابن خزيمة ، ربيع ابن  سليمان جيزي ، كان عد في حفاظ .   روى عن علي ابن بشرة سجستاني ، ذكر فراء أن توفى في رجب سنة .',
 ' فتح همزة سكون ألف فتح باء موحد سين  مهمل ساكن كاف  مضموم واو ساكن نون ، روى بعض همزة بعد باء ليس بين ألف قد  ذكر في موضع : بليد على ساحل بحر طبرستان بين بين جرجان ثلاث يوم ،  إلى نسب بحر آبسكون ، نسب إلى أبو علا

In [15]:
from tqdm import tqdm
tqdm.pandas()
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(yaqutz_pd['Clean_desc'])

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
x.toarray().sum()

70171.6165231661

In [17]:
max_values = x.max(axis=1)

In [18]:
max_values.toarray()

array([[0.52706138],
       [0.23665338],
       [0.27145979],
       ...,
       [0.44110682],
       [0.29876246],
       [0.33362928]])

In [19]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words)
word_count_vector=cv.fit_transform(stem_docs)

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['بخ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [20]:
word_count_vector.shape

(12454, 33806)

In [21]:

cv=CountVectorizer(max_df=0.85,stop_words=stop_words,max_features=10000)
word_count_vector=cv.fit_transform(stem_docs)
word_count_vector.shape

(12454, 10000)

In [22]:
list(cv.vocabulary_.keys())[:10]

['جمع', 'بئر', 'قال', 'بئار', 'أبار', 'موضع', 'أجفر', 'يد', 'خمس', 'ميل']

In [23]:
list(cv.get_feature_names())[2000:2015]

['تابع',
 'تابعي',
 'تابوت',
 'تاج',
 'تاجر',
 'تارة',
 'تارك',
 'تارم',
 'تاريخ',
 'تاسع',
 'تاشفين',
 'تالله',
 'تالي',
 'تام',
 'تامر']

In [24]:

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [25]:
tfidf_transformer.idf_

array([7.48543845, 7.2110016 , 8.03198215, ..., 7.53950567, 5.86552924,
       8.82043952])

In [26]:

feature_names=cv.get_feature_names()

In [27]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs[1]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)

# now print the results
# print("\n=====Title=====")
# print(docs_title[0])
# print("\n=====Body=====")
# print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Keywords===
كور 0.522
واسط 0.488
بئر 0.432
كورة 0.397
جمع 0.32


In [28]:
len(feature_names)

10000

In [29]:
words= set(yaqutz_pd.PlaceName.to_list())- set(feature_names)

In [31]:
places_infeature

NameError: name 'places_infeature' is not defined

In [32]:
places_infeature = set(feature_names) & set(yaqutz_pd.PlaceName.to_list())

In [33]:
import numpy as np

In [54]:
def find_word(word):
    words = [word]
    letters_s = set(words)
    
    return yaqutz_pd[yaqutz_pd.Stem_desc.str.split(' ').map(letters_s.issubset)]


In [61]:
find_word("سيح")['Description'].iloc[3]



'# بالضم ثم السكون، وضم الراء، وواو ساكنة، وسين مهملة مفتوحة، ونون، وهاء،\n<br />~~أورده أبو سعد، رحمه الله، بالسين المهملة، وهذا الذي أوردته هاهنا هو الذي\n<br />~~سمعته من ألفاظ أهل تلك البلاد: وهي بلدة كبيرة بما وراء النهر من بلاد\n<br />~~الهياطلة بين سيحون وسمرقند، وبينها وبين سمرقند ستة وعشرون فرسخا، معدودة\n<br />~~في الإقليم الرابع، طولها إحدى وتسعون درجة وسدس وعرضها ست وثلاثون درجة\n<br />~~وثلثان، قال الإصطخري: أشروسنة اسم الإقليم كما أن الصغد اسم الإقليم،\n<br />~~وليس بها مكان ولا مدينة بهذا الاسم، والغالب عليها الجبال، والذي يطوف بها\n<br />~~من أقاليم ما وراء النهر من شرقيها فرغانة، ومن غربيها حدود سمرقند،\n<br />~~وشماليها الشاش وبعض فرغانة، وجنوبيها بعض حدود كش والصغانيان وشومان\n<br />~~وواشجرد وراشت، ومدينتها الكبرى يقال لها بلسان الأشروسنة، ومن مدنها:\n<br />~~بنجيكت وساباط وزامين وديزك وخرقانة، ومدينتها التي يسكنها الولاة بنجيكت،\n<br />~~ينسب إلى أشروسنة أمم من أهل العلم منهم: أبو طلحة حكيم بن نصر بن خالج بن\n<br />~~جندبك، وقيل: جند لك الأشروسني.\n'

In [36]:
contains   =   [yaqutz_pd['Stem_desc'].str.split(' ').str.contains('سيح')]
resul = yaqutz_pd[np.all(contains, axis=0)]['Stem_desc']
print(resul)

0         جمع بئر . قال في جمع بئر بئار أبار : موضع بين...
1         فتح همزة بعد ألف باء موحد مفتوح جيم : موضع في...
2         فتح همزة سكون ألف ضم باء موحد وراء : قرية من ...
3         فتح همزة سكون ألف فتح باء موحد سين  مهمل ساكن...
4         فتح همزة بعد ألف باء مكسور لم :   أربع موضع ....
                               ...                        
12449     ضم ثم سكون ، نون بين ألف :   موضع من إلى برذع...
12450     ضم ثم سكون ثم مثل ، يوم يؤيؤ : هو يوم أوقية م...
12451     فتح ، قول تعالى : جاء قوم هرع إلى ، أي أسرع ،...
12452     نسبة إلى يهودي في موضع : أحد محلة جرجان آخر ب...
12453     فتح أول ، سكون ثاني ، ضم عين مهمل ، ثاء مثلث ...
Name: Stem_desc, Length: 12454, dtype: object


In [218]:
# _sum=0
# for x in places_infeature:
#     words = [x]
#     letters_s = set(words)
    
#     _sum+= len(yaqutz_pd[yaqutz_pd.Stem_desc.str.split(' ').map(letters_s.issubset)])
# print(_sum)

152827


In [199]:
[yaqutz_pd['Stem_desc'].str.split(' ').str.contains('سيح')]
resul = yaqutz_pd[np.all(contains, axis=0)]
resul

,PlaceName,Description,Clean_desc,Stem_desc
0,آبار الأعراب,# جمع بئر. يقال في جمعها آبار وبئار وأبار: موض...,جمع بئر . يقال في جمعها آبار وبئار وأبار : مو...,جمع بئر . قال في جمع بئر بئار أبار : موضع بين...
1,آبج,# بفتح الهمزة وبعد الألف باء موحدة مفتوحة وجيم...,بفتح الهمزة وبعد الألف باء موحدة مفتوحة وجيم ...,فتح همزة بعد ألف باء موحد مفتوح جيم : موضع في...
2,آبر,# بفتح الهمزة وسكون الألف وضم الباء الموحدة ور...,بفتح الهمزة وسكون الألف وضم الباء الموحدة ورا...,فتح همزة سكون ألف ضم باء موحد وراء : قرية من ...
3,آبسكون,# بفتح الهمزة وسكون الألف وفتح الباء الموحدة و...,بفتح الهمزة وسكون الألف وفتح الباء الموحدة وا...,فتح همزة سكون ألف فتح باء موحد سين مهمل ساكن...
4,آبل,# بفتح الهمزة وبعد الألف باء مكسورة ولام:\n<br...,بفتح الهمزة وبعد الألف باء مكسورة ولام : أر...,فتح همزة بعد ألف باء مكسور لم : أربع موضع ....
...,...,...,...,...
12449,يونان,# بالضم ثم السكون، ونونين بينهما ألف:\n<br />#...,بالضم ثم السكون ، ونونين بينهما ألف : موضع ...,ضم ثم سكون ، نون بين ألف : موضع من إلى برذع...
12450,يؤيؤ,# بالضم ثم السكون ثم مثله، يوم يؤيؤ: وهو يوم ا...,بالضم ثم السكون ثم مثله ، يوم يؤيؤ : وهو يوم ...,ضم ثم سكون ثم مثل ، يوم يؤيؤ : هو يوم أوقية م...
12451,يهرع,# بالفتح، قوله تعالى: وجاءه قومه يهرعون إليه، ...,بالفتح ، قوله تعالى : وجاءه قومه يهرعون إليه ...,فتح ، قول تعالى : جاء قوم هرع إلى ، أي أسرع ،...
12452,يهودية,# نسبة إلى اليهود في موضعين: أحدهما محلة بجرجا...,نسبة إلى اليهود في موضعين : أحدهما محلة بجرجا...,نسبة إلى يهودي في موضع : أحد محلة جرجان آخر ب...


In [169]:
df2 = yaqutz_pd['Stem_desc'].str.split(' ').copy()
'جمع' in df2

False

In [162]:
df2.apply(lambda x : [t.strip() for t in x])

0        [, جمع, بئر, ., قال, في, جمع, بئر, بئار, أبار,...
1        [, فتح, همزة, بعد, ألف, باء, موحد, مفتوح, جيم,...
2        [, فتح, همزة, سكون, ألف, ضم, باء, موحد, وراء, ...
3        [, فتح, همزة, سكون, ألف, فتح, باء, موحد, سين, ...
4        [, فتح, همزة, بعد, ألف, باء, مكسور, لم, :, , ,...
                               ...                        
12449    [, ضم, ثم, سكون, ،, نون, بين, ألف, :, , , موضع...
12450    [, ضم, ثم, سكون, ثم, مثل, ،, يوم, يؤيؤ, :, هو,...
12451    [, فتح, ،, قول, تعالى, :, جاء, قوم, هرع, إلى, ...
12452    [, نسبة, إلى, يهودي, في, موضع, :, أحد, محلة, ج...
12453    [, فتح, أول, ،, سكون, ثاني, ،, ضم, عين, مهمل, ...
Name: Stem_desc, Length: 12454, dtype: object